In [16]:
import re

"""
<Request Host> - [<Start Date>] "<Request Method> <Request URL> <Request Protocol>" <Status Code> <Bytes Received> <Bytes Sent> "<Referer>" "<User-Agent>" <Remote Address> 
x_forwarded_for:"<X-Forwarded-For>" x_forwarded_proto:"<X-Forwarded-Proto>" vcap_request_id:<X-Vcap-Request-ID> response_time:<Response Time> app_id:<Application ID> <Extra Headers>
""";

lgPattern = "\<(\d+)\>([^ ]+) ([^ ]+) ([^ ]+) \[([^\]]+)+\]"

#                      hostname   time             request   
goRouterBasePattern = '([^ ]+) - \[([0-9T:.+-]+)\] "(.+ HTTP\/1\.1)" "?([\d-]+)"? (\d+) (\d+) "([^"]+)" "([0-9\.]+):([\d]+)" "([0-9\.]+):([\d]+)" "([^"]+)"'
goRouterKvPattern = '[a-z_]+:"([^"]+)" [a-z_]+:"([^"]+)" [a-z_]+:"([^"]+)" [a-z_]+:([^ ]+) [a-z_]+:([^ ]+) (.*)'


pat = lgPattern + "  " + goRouterBasePattern + " " + goRouterKvPattern
grtest = '([^ ]+) - '

lSample = "<6>2017-04-21T09:15:43.107185+00:00 10.10.81.6 vcap.gorouter [job=router_z2 index=2]"

fullsample = """\
<6>2017-04-21T09:15:43.107185+00:00 10.10.81.6 vcap.gorouter [job=router_z2 index=2]  login.run.pivotal.io - [2017-04-21T09:15:43.089+0000] "POST /oauth/token HTTP/1.1" 200 29 1549 "-" "Ruby" "10.10.66.114:56583" "10.10.81.30:8080" x_forwarded_for:"54.210.10.202" x_forwarded_proto:"https" vcap_request_id:"dc80201f-29c8-44cb-6c8e-ddc2c4f7bcd6" response_time:0.018057238 app_id:"-" app_index:"-" x_b3_traceid:"c2b37a87dbf533ae" x_b3_spanid:"c2b37a87dbf533ae" x_b3_parentspanid:"-"
"""

fs2 = """\
<6>2017-04-26T00:03:18.295812+00:00 10.10.81.9 vcap.gorouter [job=router_z2 index=3]  dogate-tracker-aws.cfapps.io - [2017-04-26T00:03:18.266+0000] "POST /api/v1/series?api_key=80adce40484093449d60d6e76a5dae92 HTTP/1.1" 200 292 0 "-" "Ruby" "10.10.66.97:42773" "10.10.148.96:61013" x_forwarded_for:"52.89.159.192" x_forwarded_proto:"https" vcap_request_id:"bc4fb926-ee40-4aaa-7963-b7d8616d8b36" response_time:0.029437206 app_id:"6d13de12-6479-476e-b4a8-b7aa0d98127a" app_index:"0" x_b3_traceid:"80939612ff51224d" x_b3_spanid:"80939612ff51224d" x_b3_parentspanid:"-"
"""

fs3 =\
"""\
<6>2017-05-02T10:39:13.984285+00:00 10.10.81.5 vcap.gorouter [job=router_z2 index=4]  portal.europeana.eu - [2017-05-02T10:39:13.973+0000] "GET /portal/record/2020704/DR_20284776.html HTTP/1.1" 302 0 139 "-" "Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)" "10.10.66.198:22939" "10.10.148.100:61108" x_forwarded_for:"68.180.230.153, 54.145.66.20" x_forwarded_proto:"http" vcap_request_id:"35244aa9-8cf3-405f-502e-0b74a052b43b" response_time:0.010386265 app_id:"4ee25a7d-9b17-49e0-9a82-8a1494337597" app_index:"4" x_b3_traceid:"e5b06239f0e6dde7" x_b3_spanid:"e5b06239f0e6dde7" x_b3_parentspanid:"-"
"""

fs4 = """\
<6>2017-04-23T10:17:40.962755+00:00 10.10.17.2 vcap.gorouter [job=router_z1 index=0]  powerpuffgirls.cfapps.io:4443 - [2017-04-23T10:17:40.933+0000] "GET /api/v3/users/websocket HTTP/1.1" "-" 0 0 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36" "10.10.66.97:59790" "10.10.148.147:61048" x_forwarded_for:"10.10.66.97" x_forwarded_proto:"-" vcap_request_id:"356f5ff1-9ae3-4cc6-4b5d-b848dd7b61dd" response_time:0.029583438 app_id:"8532d3f3-18a7-4f67-9481-57431aed7748" app_index:"0" x_b3_traceid:"76939e8c901442b4" x_b3_spanid:"76939e8c901442b4" x_b3_parentspanid:"-"
"""

sample = """\
<6>2017-04-23T10:17:40.962755+00:00 10.10.17.2 vcap.gorouter [job=router_z1 index=0]  login.run.pivotal.io - [2017-04-21T09:15:43.089+0000] "POST /oauth/token HTTP/1.1" 200 29 1549 "-" "Ruby" "10.10.66.114:56583" "10.10.81.30:8080" x_forwarded_for:"54.210.10.202" x_forwarded_proto:"https" vcap_request_id:"dc80201f-29c8-44cb-6c8e-ddc2c4f7bcd6" response_time:0.018057238 app_id:"-" app_index:"-" x_b3_traceid:"c2b37a87dbf533ae" x_b3_spanid:"c2b37a87dbf533ae" x_b3_parentspanid:"-"
"""

In [17]:
re.match(pat, fs4).groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [3]:
(level, time, srcip, srcprog, progdetail,\
reqhost, reqstart, reqmethodAndurl, reqproto, statuscode,\
bytesrecv, bytessent, referer, uagent, remaddr, xfwdfr, xfwdp, 
vcapreq, resptime, appid, extra) =\
re.match(lgPattern + "  " + goRouterBasePattern + " " + goRouterKvPattern, 
         fullsample).groups()

In [14]:
m = re.compile(pat)

In [18]:
fs2 = """
<6>2017-04-26T00:03:18.295812+00:00 10.10.81.9 vcap.gorouter [job=router_z2 index=3]  dogate-tracker-aws.cfapps.io - [2017-04-26T00:03:18.266+0000] "POST /api/v1/series?api_key=80adce40484093449d60d6e76a5dae92 HTTP/1.1" 200 292 0 "-" "Ruby" "10.10.66.97:42773" "10.10.148.96:61013" x_forwarded_for:"52.89.159.192" x_forwarded_proto:"https" vcap_request_id:"bc4fb926-ee40-4aaa-7963-b7d8616d8b36" response_time:0.029437206 app_id:"6d13de12-6479-476e-b4a8-b7aa0d98127a" app_index:"0" x_b3_traceid:"80939612ff51224d" x_b3_spanid:"80939612ff51224d" x_b3_parentspanid:"-"
"""

fs3 =\
"""\
<6>2017-05-02T10:39:13.984285+00:00 10.10.81.5 vcap.gorouter [job=router_z2 index=4]  portal.europeana.eu - [2017-05-02T10:39:13.973+0000] "GET /portal/record/2020704/DR_20284776.html HTTP/1.1" 302 0 139 "-" "Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)" "10.10.66.198:22939" "10.10.148.100:61108" x_forwarded_for:"68.180.230.153, 54.145.66.20" x_forwarded_proto:"http" vcap_request_id:"35244aa9-8cf3-405f-502e-0b74a052b43b" response_time:0.010386265 app_id:"4ee25a7d-9b17-49e0-9a82-8a1494337597" app_index:"4" x_b3_traceid:"e5b06239f0e6dde7" x_b3_spanid:"e5b06239f0e6dde7" x_b3_parentspanid:"-"
"""

m.match(fs2).groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [120]:
%%timeit n=100
m.match(ful);

100000 loops, best of 3: 2.41 µs per loop


```
create table router (
level smallint,
time timestamp,
srcip inet,
srcprog text,
progdetail text,
reqhost text,
reqstart timestamp,
reqmethodAndurl text,
reqproto text,
statuscode int,
bytesrecv bigint,
bytessent text,
referer text,
uagent text,
remaddr text,
xfwdfr text,
xfwdp text,
vcapreq text,
resptime float,
appid text,
extra text
) with (appendonly=true, compresstype=zlib, compresslevel=5, orientation=column)
distributed randomly;
```

```
30177 in 22min - 1 thread
30k in 3.5 min - 1 thread
30k in 3.3 min - 1 thread (only 24 edge cases - payload overrun) - 15MB raw (3MB loaded)
150k in 9.3 min - 1 thread  (only 89 edge cases)
```

```
cat /Users/kdunn/Desktop/rparsed.csv | psql -h 172.16.139.132 -U gpadmin -c "COPY router from stdin CSV "
```

```
<6>2017-05-02T10:49:41.247897+00:00 10.10.17.5 vcap.gorouter [job=router_z1 index=4]  cloudmetricsservice.cfapps.io - [2017-05-02T10:49:41.241+0000] "GET /api/list/metrics/customer/312/account/274/resource/8a8afb40-0de6-466d-bcd1-63e48a4874e6-ccirsglivitsnl-ccistossdvmc001/metric/274/period/26/min HTTP/1.1" 200 0 2 "-" "Apache-HttpClient/4.3.6 (java 1.5)" "10.10.66.97

<6>2017-05-02T10:42:47.109925+00:00 10.10.81.6 vcap.gorouter [job=router_z2 index=2]  www.europeana.eu - [2017-05-02T10:42:47.077+0000] "GET /portal/record/09431/view_show_photos_pl_id_99958_SC_id_RC_1_SRV_true_LANG_bg_SA_99958.html HTTP/1.1" 302 0 179 "-"
```